<a href="https://colab.research.google.com/github/jbliaw/QM323/blob/main/AutomatingConjointRegressions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- NOTE: Please make a copy of this file for yourself (it's like a google doc). That way you can edit as you need, and it won't interfere with other runs. 
- Before you start, prep the data for regression in excel (basically Conjoint Tutorial as posted in Conjoint Workshop QM 11 folder): 
     - transpose direct data from Qualtrics (.csv file)
     - download "dummy variables for conjoint analysis regressions" on QTools (.xlsx file)
     - copy and paste customers to "dummy variables..." excel file (see JetOrange example) (keep as .xlsx file)
          - make sure you name each customer as Customer1, Customer2 instead of 'Customer 1', 'Customer 2', etc. (omit the space)
          - make sure your conjoint survey data is on the first sheet of the excel workbook (there's a way to select certain sheets using the pandas library but I haven't included it in this code. If you're interested, simply google 'pandas read specific excel sheet' and the docs should come up. Also, .csv files, though they look like .xlsx files, don't actually save anything other than the first sheet, so be careful to not lose your work by saving it as a .csv)
     - rename file to 'conjointdata.xlsx' so that you can upload to Google Colab
- Then upload the file: 
     - click folder icon to the left
     - right click in white space (or regular click page with arrow icon) and upload 'conjointdata.xlsx'
     - click each 'play'/run button (more instructions as #comments in each code block)

## General Prep

In [ ]:
# Necessary imports
import pandas as pd
import pandas.testing as tm
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
# Read in data
df = pd.read_excel('conjointdata.xlsx')
print(df.shape) # (rows, columns)
df.head() # Take a look at the data

(9, 49)


,Profile,Price high,Price low,atribute 2 high,attribute 3 high,attribute 4 high,Customer1,Customer2,Customer3,Customer4,Customer5,Customer6,Customer7,Customer8,Customer9,Customer10,Customer11,Customer12,Customer13,Customer14,Customer15,Customer16,Customer17,Customer18,Customer19,Customer20,Customer21,Customer22,Customer23,Customer24,Customer25,Customer26,Customer27,Customer28,Customer29,Customer30,Customer31,Customer32,Customer33,Customer34,Customer35,Customer36,Customer37,Customer38,Customer39,Customer40,Customer41,Customer42,Customer43
0,1,1,0,1,1,0,5,3,5,6,3,5,4,6,7,1,3,5,4,1,4,4,2,5,5,2,3,6,1,6,6,4,5,4,6,4,3,5,3,6,7,2,4,4,7,5,3,2,6
1,2,0,0,0,0,1,5,3,5,6,3,5,3,2,5,3,1,3,7,7,4,4,7,5,1,7,2,4,7,4,4,5,4,3,6,5,3,6,2,2,4,1,3,6,5,3,2,2,3
2,3,0,0,1,1,1,5,3,5,6,5,7,2,6,7,5,3,3,5,5,3,6,7,2,7,2,6,5,3,5,3,3,5,7,6,6,2,3,2,5,7,1,6,5,5,2,2,2,4
3,4,1,0,0,1,1,5,1,5,6,4,5,2,2,4,1,1,3,5,6,3,6,7,3,2,7,2,4,3,5,4,3,4,4,7,4,2,5,2,1,4,1,6,5,5,3,3,2,3
4,5,1,0,1,0,0,5,1,5,6,4,4,3,6,5,1,3,4,6,1,4,4,2,7,6,2,6,6,1,4,5,6,5,2,4,4,5,2,3,6,7,3,4,6,6,5,3,2,6


# Short Way

In [ ]:
# Defining a function to do it all at once

def automateregression(df):
  # Rename columns so they're easier to grab
  df = df.rename(columns={"Price high": "pricehigh", 
                   "Price low": "pricelow", 
                   "atribute 2 high": "attribute2high",
                   "attribute 3 high": "attribute3high", 
                   "attribute 4 high": "attribute4high"}, errors='raise')
  
  # Create a list of customers to iterate through
  listofcustomers = []
  for (columns, columndata) in df.iteritems():
    if columns.startswith("Customer"):
      listofcustomers.append(columns)
    else: continue

  # Create an empty dataframe (theresults) with proper column names
  theresults = pd.DataFrame(columns = ['Intercept', 'pricehigh', 'pricelow', 'attribute2high', 'attribute3high', 'attribute4high'])

  # Iterating thru each customer and running regressions
  for eachcustomer in listofcustomers:
    customer = eachcustomer
    # Run regressions
    mod = smf.ols(formula = '{} ~ pricehigh + pricelow + attribute2high + attribute3high + attribute4high'.format(customer), data = df)
    res = mod.fit()
    results_summary = res.summary() # Regression results
    results_as_html = results_summary.tables[1].as_html() # Converting results summary into dataframe
    newdf = pd.read_html(results_as_html, header=0, index_col=0)[0] # Converting results summary into dataframe
    transposed = newdf.T # Transposing to new dataframe
    theresults = theresults.append(transposed.iloc[0]) # Appending transposed dataframe to theresults

  # Resetting index for nice formatting
  theresults = theresults.reset_index(drop=True)
  theresults.index = theresults.index + 1

  return theresults

In [ ]:
# Run the function with your data (df)
theresults = automateregression(df)
# Ignore big block of text, it's not important in our case

# Sve theresults as an excel file
theresults.to_excel('conjointresults.xlsx')

# TO DOWNLOAD: 
#      - Click folder icon to the left, file named 'conjointresults.xlsx' should appear
#      - Right click the file and download to your computer

/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:1535: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  "anyway, n=%i" % int(n))
/usr/local/lib/python3.6/dist-packages/statsmodels/regression/linear_model.py:1636: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:1535: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  "anyway, n=%i" % int(n))
/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:1535: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  "anyway, n=%i" % int(n))
/usr/local/lib/python3.6/dist-packages/statsmodels/regression/linear_model.py:1636: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:1535: UserWarning: kurtosistest only valid for n>=20 ... continuing a

# Long Way (if you're interested in seeing/understanding how it works step by step)

In [ ]:
# Read in data
df1 = pd.read_excel('conjointdata.xlsx')
print(df1.shape) # (rows, columns)
df1.head() # Take a look at the data

(9, 49)


,Profile,Price high,Price low,atribute 2 high,attribute 3 high,attribute 4 high,Customer1,Customer2,Customer3,Customer4,Customer5,Customer6,Customer7,Customer8,Customer9,Customer10,Customer11,Customer12,Customer13,Customer14,Customer15,Customer16,Customer17,Customer18,Customer19,Customer20,Customer21,Customer22,Customer23,Customer24,Customer25,Customer26,Customer27,Customer28,Customer29,Customer30,Customer31,Customer32,Customer33,Customer34,Customer35,Customer36,Customer37,Customer38,Customer39,Customer40,Customer41,Customer42,Customer43
0,1,1,0,1,1,0,5,3,5,6,3,5,4,6,7,1,3,5,4,1,4,4,2,5,5,2,3,6,1,6,6,4,5,4,6,4,3,5,3,6,7,2,4,4,7,5,3,2,6
1,2,0,0,0,0,1,5,3,5,6,3,5,3,2,5,3,1,3,7,7,4,4,7,5,1,7,2,4,7,4,4,5,4,3,6,5,3,6,2,2,4,1,3,6,5,3,2,2,3
2,3,0,0,1,1,1,5,3,5,6,5,7,2,6,7,5,3,3,5,5,3,6,7,2,7,2,6,5,3,5,3,3,5,7,6,6,2,3,2,5,7,1,6,5,5,2,2,2,4
3,4,1,0,0,1,1,5,1,5,6,4,5,2,2,4,1,1,3,5,6,3,6,7,3,2,7,2,4,3,5,4,3,4,4,7,4,2,5,2,1,4,1,6,5,5,3,3,2,3
4,5,1,0,1,0,0,5,1,5,6,4,4,3,6,5,1,3,4,6,1,4,4,2,7,6,2,6,6,1,4,5,6,5,2,4,4,5,2,3,6,7,3,4,6,6,5,3,2,6


In [ ]:
# Rename columns so they're easier to grab
df1 = df1.rename(columns={"Price high": "pricehigh", 
                   "Price low": "pricelow", 
                   "atribute 2 high": "attribute2high",
                   "attribute 3 high": "attribute3high", 
                   "attribute 4 high": "attribute4high"}, errors='raise')
df1.head()

,Profile,pricehigh,pricelow,attribute2high,attribute3high,attribute4high,Customer1,Customer2,Customer3,Customer4,Customer5,Customer6,Customer7,Customer8,Customer9,Customer10,Customer11,Customer12,Customer13,Customer14,Customer15,Customer16,Customer17,Customer18,Customer19,Customer20,Customer21,Customer22,Customer23,Customer24,Customer25,Customer26,Customer27,Customer28,Customer29,Customer30,Customer31,Customer32,Customer33,Customer34,Customer35,Customer36,Customer37,Customer38,Customer39,Customer40,Customer41,Customer42,Customer43
0,1,1,0,1,1,0,5,3,5,6,3,5,4,6,7,1,3,5,4,1,4,4,2,5,5,2,3,6,1,6,6,4,5,4,6,4,3,5,3,6,7,2,4,4,7,5,3,2,6
1,2,0,0,0,0,1,5,3,5,6,3,5,3,2,5,3,1,3,7,7,4,4,7,5,1,7,2,4,7,4,4,5,4,3,6,5,3,6,2,2,4,1,3,6,5,3,2,2,3
2,3,0,0,1,1,1,5,3,5,6,5,7,2,6,7,5,3,3,5,5,3,6,7,2,7,2,6,5,3,5,3,3,5,7,6,6,2,3,2,5,7,1,6,5,5,2,2,2,4
3,4,1,0,0,1,1,5,1,5,6,4,5,2,2,4,1,1,3,5,6,3,6,7,3,2,7,2,4,3,5,4,3,4,4,7,4,2,5,2,1,4,1,6,5,5,3,3,2,3
4,5,1,0,1,0,0,5,1,5,6,4,4,3,6,5,1,3,4,6,1,4,4,2,7,6,2,6,6,1,4,5,6,5,2,4,4,5,2,3,6,7,3,4,6,6,5,3,2,6


In [ ]:
# Create a list of customers to iterate through
listofcustomers = []
for (columns, columndata) in df1.iteritems():
  if columns.startswith("Customer"):
    listofcustomers.append(columns)
  else: continue
print(listofcustomers) # Take a look at the list

['Customer1', 'Customer2', 'Customer3', 'Customer4', 'Customer5', 'Customer6', 'Customer7', 'Customer8', 'Customer9', 'Customer10', 'Customer11', 'Customer12', 'Customer13', 'Customer14', 'Customer15', 'Customer16', 'Customer17', 'Customer18', 'Customer19', 'Customer20', 'Customer21', 'Customer22', 'Customer23', 'Customer24', 'Customer25', 'Customer26', 'Customer27', 'Customer28', 'Customer29', 'Customer30', 'Customer31', 'Customer32', 'Customer33', 'Customer34', 'Customer35', 'Customer36', 'Customer37', 'Customer38', 'Customer39', 'Customer40', 'Customer41', 'Customer42', 'Customer43']


In [ ]:
# Create an empty dataframe with proper column names
theresults = pd.DataFrame(columns = ['Intercept', 'pricehigh', 'pricelow', 'attribute2high', 'attribute3high', 'attribute4high'])
print(theresults)

Empty DataFrame
Columns: [Intercept, pricehigh, pricelow, attribute2high, attribute3high, attribute4high]
Index: []


In [ ]:
# Iterating thru each customer and running regressions
for eachcustomer in listofcustomers:
  customer = eachcustomer
  mod = smf.ols(formula = '{} ~ pricehigh + pricelow + attribute2high + attribute3high + attribute4high'.format(customer), data = df1)
  res = mod.fit()
  results_summary = res.summary()
  results_as_html = results_summary.tables[1].as_html() # Converting results summary into dataframe
  newdf = pd.read_html(results_as_html, header=0, index_col=0)[0] # Converting results summary into dataframe
  transposed = newdf.T
  theresults = theresults.append(transposed.iloc[0])
#theresults #(remove the first # from this line if you want to see the results)
# uhhh, ignore the big block of text, it's not important in our case

/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:1535: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  "anyway, n=%i" % int(n))
/usr/local/lib/python3.6/dist-packages/statsmodels/regression/linear_model.py:1636: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:1535: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  "anyway, n=%i" % int(n))
/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:1535: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  "anyway, n=%i" % int(n))
/usr/local/lib/python3.6/dist-packages/statsmodels/regression/linear_model.py:1636: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:1535: UserWarning: kurtosistest only valid for n>=20 ... continuing a

In [ ]:
theresults.shape # 43 responses, 6 columns for intercept, pricelow, pricehigh, at2, at3, at4

(43, 6)

In [ ]:
# Reformatting the index
theresults = theresults.reset_index(drop=True)
theresults.index = theresults.index + 1
theresults

,Intercept,pricehigh,pricelow,attribute2high,attribute3high,attribute4high
1,5.0000,2.220000e-16,4.441000e-16,1.776000e-15,1.998000e-15,1.998000e-15
2,1.3333,-3.333000e-01,-1.000000e+00,3.333000e-01,3.333000e-01,6.667000e-01
3,5.0000,2.220000e-16,4.441000e-16,1.776000e-15,1.998000e-15,1.998000e-15
4,6.0000,2.776000e-17,1.332000e-15,2.665000e-15,2.665000e-15,1.998000e-15
5,4.6667,-1.666700e+00,1.000000e+00,6.667000e-01,6.667000e-01,-6.667000e-01
6,2.3500,-1.500000e-01,-2.500000e-01,1.800000e+00,8.000000e-01,2.200000e+00
7,4.6667,-6.667000e-01,1.110000e-15,-3.333000e-01,-3.333000e-01,-1.666700e+00
8,2.5333,-1.333000e-01,2.000000e+00,2.933300e+00,9.333000e-01,-9.333000e-01
9,3.2833,1.167000e-01,-2.500000e-01,9.333000e-01,1.933300e+00,6.670000e-02
10,3.2000,-3.300000e+00,1.500000e+00,6.000000e-01,1.100000e+00,-1.000000e-01


In [ ]:
# Download to an excel file
theresults.to_excel('conjointresults.xlsx')